In [9]:
! pip install pydot

In [10]:
! pip install textX

In [8]:
#To make a meta-model from a file
from textx import metamodel_from_file

#To make the java files
import os

In [4]:
#Make the meta-model
metamodel = metamodel_from_file('EBNF.tx')

In [5]:
#Create the syntax tree
syntaxTree = metamodel.model_from_file('fullExample.tx')

In [20]:
def to_java(model, output_dir):
    
    #Internal methods to make it more organized heeheh
    #Change the diagrams visibility to actual java visibilities
    def visToJava(vis):
        mapping = {
            '+': 'public',
            '-': 'private',
            '#': 'protected'
        }
        return mapping[vis]
    
    #Organize an attribute in java format
    def attributeToJava(att):
         return f"\t{visToJava(att.visibility.visibility)} {att.type.datatype} {att.name.lowName};"
    
    #Manage the appropiate return types
    def defaultReturnValue(datatype):
        mapping = {
            "int": "return 0;",
            "double": "return 0.0;",
            "long": "return 0L;",
            "float": "return 0.0f;",
            "char": "return '\u0000';",
            "boolean": "return false;",
            "void": ""
        }
        return mapping.get(datatype, "return null;")

    
    #Organize a method in java format
    def methodToJava(met):
        if met.params:
            params_str = ', '.join(f"{param.type.datatype} {param.name.lowName}" for param in met.params.params)
        else:
            params_str = ''

        return_type = met.returnType.retorno.datatype
        return_value = defaultReturnValue(return_type)

        return f"\t{visToJava(met.visibility.visibility)} {return_type} {met.name.lowName}({params_str}) {{ {return_value} }}"
    
    #Store all the relations in a key map so then all can be called later for each class
    def relationsToJava():
        result = {}

        for rel in model.relation:
            start = rel.start.upName
            end = rel.end.upName
            multiplicity = rel.endMultiplicity.multiplicities

            if multiplicity.strip() in ['0..*', '1..*']:
                type_str = f"List<{end}>"
            else:
                type_str = end

            rel_name = end[0].lower() + end[1:]

            java_attr = f"\tprivate {type_str} {rel_name};"

            if start not in result:
                result[start] = []
            result[start].append(java_attr)

        return result

    #To get list of relations of a class
    def getRelations(className):
        return formattedRelations.get(className, [])
    
    #ACTUAL PROCESS ------------------------------------------------
    
    formattedRelations = relationsToJava()

    for obj in model.objects:
        
        class_obj = obj.objectType

        name = class_obj.name.upName

        attributes = class_obj.attributes

        methods = class_obj.methods

        for obj in model.objects:
            lines = []

            print("import java.util.List;")
            lines.append("import java.util.List;")
            
            header = f"public class {name} {{"
            print(header)
            lines.append(header)

            print("\n\t//Attributes")
            lines.append("\t//Attributes")
            for att in attributes:
                line = attributeToJava(att)
                print(line)
                lines.append(line)

            print("\n\t//Methods")
            lines.append("\n\t//Methods")
            for met in methods:
                line = methodToJava(met)
                print(line)
                lines.append(line)

            print("\n\t//Relations")
            lines.append("\n\t//Relations")
            for relation in getRelations(name):
                print(relation)
                lines.append(relation)
                
            print("}\n")
            lines.append("}\n")

            # Write to file
            file_path = os.path.join(output_dir, f"{name}.java")
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(lines))

In [21]:
to_java(syntaxTree, "java files")

import java.util.List;
public class Persona {

	//Attributes
	public String nombre;
	public int edad;
	public float altura;

	//Methods
	public String hablar() { return null; }
	public void caminar() {  }
	public void conducir(String carro, int velocidad) {  }

	//Relations
	private List<Carro> carro;
	private Licencia licencia;
	private Conductor conductor;
	private List<Empresa> empresa;
}

import java.util.List;
public class Persona {

	//Attributes
	public String nombre;
	public int edad;
	public float altura;

	//Methods
	public String hablar() { return null; }
	public void caminar() {  }
	public void conducir(String carro, int velocidad) {  }

	//Relations
	private List<Carro> carro;
	private Licencia licencia;
	private Conductor conductor;
	private List<Empresa> empresa;
}

import java.util.List;
public class Persona {

	//Attributes
	public String nombre;
	public int edad;
	public float altura;

	//Methods
	public String hablar() { return null; }
	public void caminar() {  }
	pu

In [16]:
!textx generate EBNF.tx --target dot

Generating dot target from models:
c:\Users\kracr\OneDrive\Desktop\Discretas3\Context Free Grammars\EBNF.tx
-> c:\Users\kracr\OneDrive\Desktop\Discretas3\Context Free Grammars\EBNF.dot
     To convert to png run "dot -Tpng -O EBNF.dot"


In [20]:
import pydot
graph = pydot.graph_from_dot_file('EBNF.dot')[0]
graph.write_png('intento.png')

FileNotFoundError: [WinError 2] "dot" not found in path.